We want the neural net to be given the same size volume at each episode.



First, we upsample the SWC files to make sure to get rid of huge inter-node distances, which can cause problems when trying to render a volume.

Before running the `resample_swc` script, this are the steps that Yang took:

He went to 'v3d_tools/release_plugins/v3d_plugins/resample_swc' and ran `qmake` and then `make`. He made some symbolic links to the main `v3d_external` folder.

Then, to run the `resample_swc` script, simply run `/path/to/v3d_external/bin/vaa3d -x resample_swc -f resample_swc -i <input_file> -o <output_file>`.

We need to get the resample_swc file directory

we check to see if the branch is at least as big as the target volume before we sample nodes from it.

I tried minimum dimension cut-off of 25, but no branches met that criteria.

In [5]:
from swc_io import get_fnames_and_abspath_from_dir, swc_to_dframe, resample_swc, dframe_to_swc
import numpy as np
import pandas as pd

BRANCHES_DIR = "../data/03_human_branches_splitted"
UPSAMPLED_BRANCHES_DIR = "../data/04_human_branches_filtered_upsampled"

Specify the path to the Vaa3D compiled binary: "/path/to/v3d_external/bin/vaa3d"

In [6]:
import tkinter as tk
from tkinter import filedialog

root = tk.Tk()
root.withdraw()

vaa3d_path = filedialog.askopenfilename(title='Select compiled Vaa3d binary')


I used concurrent.futures to split the job over multiple processors, change `n_workers` to suit your needs.

In [7]:
from threader import multithreading

Use the "resample_swc" Vaa3D plugin (written in C) to upsample all our data. 

In [8]:
fnames, abs_paths = get_fnames_and_abspath_from_dir(BRANCHES_DIR)
assert(len(fnames)>0)

vaa3d_repeated = [vaa3d_path for _ in range(len(fnames))]

multithreading(resample_swc, zip(fnames, abs_paths, vaa3d_repeated), 16)

['/home/ubuntu/Desktop/vaa3d_tools/hackathon/profundo/data/04_human_branches_filtered_upsampled/041_YJ_LijLiu_05172018.ano.swc_root6487578_child6487577.swc',
 '/home/ubuntu/Desktop/vaa3d_tools/hackathon/profundo/data/04_human_branches_filtered_upsampled/103_ZSJ_LijLiu_06072018.ano.swc_root2219778_child2219777.swc',
 '/home/ubuntu/Desktop/vaa3d_tools/hackathon/profundo/data/04_human_branches_filtered_upsampled/024_ZSJ_LijLiu_06122018.ano.swc_root1211834_child1211833.swc',
 '/home/ubuntu/Desktop/vaa3d_tools/hackathon/profundo/data/04_human_branches_filtered_upsampled/112_YJ_LijLiu05242018.ano.swc_root4061338_child4061337.swc',
 '/home/ubuntu/Desktop/vaa3d_tools/hackathon/profundo/data/04_human_branches_filtered_upsampled/034_YJ_LijLiu05302018.ano.swc_root1211843_child1211842.swc',
 '/home/ubuntu/Desktop/vaa3d_tools/hackathon/profundo/data/04_human_branches_filtered_upsampled/113_HGD_LijLiu_05302018.ano.swc_root6843788_child6843790.swc',
 '/home/ubuntu/Desktop/vaa3d_tools/hackathon/profun

Next, we want to only randomly sample from nodes sufficiently far away from the boundary. 

In [10]:
# tested size 25, but no branches met this criteria
TARGET_DIM_SIZE = 15  # size in pixels for all x,y,z dims
half_size = TARGET_DIM_SIZE / 2
#min_swc_dim_span = TARGET_DIM_SIZE + 5  # some leeway to make things easy
padding = TARGET_DIM_SIZE//2

print("filtering branches that have any dimension smaller than ", TARGET_DIM_SIZE)

fnames, abs_paths = get_fnames_and_abspath_from_dir(UPSAMPLED_BRANCHES_DIR)
assert(len(fnames)>0)


def upsampled_branches_to_cubes(swc_fname, swc_abs_path):
    # load to dframe
    nodes = swc_to_dframe(swc_abs_path)
        
    assert isinstance(nodes, pd.DataFrame)
    
    x_span = nodes.x.max() - nodes.x.min()
    y_span = nodes.y.max() - nodes.y.min()
    z_span = nodes.z.max() - nodes.z.min()
    
    dim_size_good = np.array([x_span, y_span, z_span]) >= TARGET_DIM_SIZE
    if np.all(dim_size_good):  # our SWC meets our criteria
        
        # find nodes far enough away from bounds
        central_nodes = nodes.loc[(nodes.x.between(nodes.x.min()+padding,nodes.x.max()+padding)) &
                              (nodes.y.between(nodes.y.min()+padding,nodes.y.max()+padding)) &
                              (nodes.z.between(nodes.z.min()+padding,nodes.z.max()+padding))]
        
        n_central_nodes = len(central_nodes.index)
        if n_central_nodes > 0:
            # TODO: smarter sampling
            sampled_nodes = central_nodes.sample(n=int(n_central_nodes/50), replace=False)

            for sample_i, center_node in sampled_nodes.iterrows():
                    # grab all nodes in neighborhood of center_node
                    sample = nodes.loc[(nodes.x.between(center_node.x.item()-half_size, center_node.x.item()+half_size)) &
                                      (nodes.y.between(center_node.y.item()-half_size, center_node.y.item()+half_size)) &
                                      (nodes.z.between(center_node.z.item()-half_size, center_node.z.item()+half_size))]

#                     print(sample)

                    # save sample to swc
                    print("saving ", fnames[sample_i])
                    dframe_to_swc(swc_fname+"sample{}".format(sample_i), sample,
                                output_dir="../data/05_sampled_cubes/")
    else:  # SWC does not have large enough dims
        #print("dims too small ", x_span, y_span, z_span) 
        pass
    
_ = multithreading(upsampled_branches_to_cubes, zip(fnames, abs_paths), 16)
    

filtering branches that have any dimension smaller than  15
saving  054_06102018.ano.swc_root5156192_child5156191.swc
saving  115_LijLiu_05272018.ano.swc_root3829157_child3829156.swc
saving  040_06102018.ano.swc_root1027271_child1027270.swc
saving  080_05092018.ano.swc_root1259191_child1259190.swc
saving saving  112_YJ_LijLiu05242018.ano.swc_root5586545_child5586544.swc 
112_YJ_LijLiu05242018.ano.swc_root5587007_child5587006.swc
saving  065_06112018.ano.swc_root4849669_child4849668.swc
saving  040_06102018.ano.swc_root1053554_child1053553.swc
saving  103_ZSJ_LijLiu_06122018.ano.swc_root1683758_child1683757.swc
saving  039_YJ_LijLiu_05192018.ano.swc_root12145325_child12145324.swc
saving  112_YJ_LijLiu05242018.ano.swc_root5585229_child5585228.swc
saving  036_HGD_LijLiu_HGD_05232018.ano.swc_root970679_child970678.swc
saving  079_06112018.ano.swc_root7043142_child7043141.swc
saving  118_HGD_LijLiu_05302018.ano.swc_root2002694_child2002698.swc
saving saving  036_HGD_LijLiu_HGD_05232018.ano.

saving  049_YJ_LijLiu_05242018.ano.swc_root2313318_child2313317.swc
saving  041_YJ_LijLiu_05172018.ano.swc_root6497846_child6497845.swc
saving  005_LijLiu_05272018.ano.swc_root1375217_child1375216.swc
saving  048_06102018.ano.swc_root1454377_child1454376.swc
saving  112_YJ_LijLiu05242018.ano.swc_root5587007_child5587006.swc
saving  024_ZSJ_LijLiu_06032018.ano.swc_root689935_child689934.swc
saving  078_LiuAn_LiujLiu05282018.ano.swc_root5036468_child5036467.swc
saving  037_06082018.ano.swc_root1173751_child1173750.swc
saving  049_YJ_LijLiu_05242018.ano.swc_root3195032_child3195031.swc
saving  064_06112018.ano.swc_root4139088_child4139087.swc
saving  037_06082018.ano.swc_root1177320_child1177319.swc
saving  052_06102018.ano.swc_root1927758_child1927757.swc
saving saving  074_LiuAn_LijLiu_Liuan_05152018.ano.swc_root3772396_child3772395.swc 037_06082018.ano.swc_root1174014_child1174013.swc

saving  075_WS3_LijLiu_Liuan_05152018.ano.swc_root4050697_child4050696.swc
saving  080_05092018.ano.s

saving  117_ZZ_LijLiu05212018.ano.swc_root26642821_child26642820.swc
saving  103_ZSJ_LijLiu_06122018.ano.swc_root1683758_child1683757.swc
saving  024_ZSJ_LijLiu_06122018.ano.swc_root1204737_child1204736.swc
saving  040_06102018.ano.swc_root1085840_child1085839.swc
saving  103_ZSJ_LijLiu_06072018.ano.swc_root2210065_child2210064.swc
saving  080_05092018.ano.swc_root1723066_child1723065.swc
saving  065_06112018.ano.swc_root4789984_child4789983.swc
saving  025_ZSJ_LijLiu_06072018.ano.swc_root5306575_child5306574.swc
saving  117_ZZ_LijLiu05212018.ano.swc_root26643192_child26643191.swc
saving  037_06082018.ano.swc_root1183003_child1183002.swc
saving  040_06102018.ano.swc_root1027271_child1027270.swc
saving  065_06112018.ano.swc_root4849669_child4849668.swc
saving  116_Linus_LijLiu_06042018.ano.swc_root4596935_child4596934.swc
saving  053__06102018.ano.swc_root3877783_child3877782.swc
saving  037_06082018.ano.swc_root1173379_child1173378.swc
saving  024_ZSJ_LijLiu_06122018.ano.swc_root119570

saving  040_06102018.ano.swc_root1027271_child1027270.swc
saving  034_YJ_LijLiu05302018.ano.swc_root1198820_child1198819.swc
saving  025_ZSJ_LijLiu_06072018.ano.swc_root5036033_child5036032.swc
saving  065_06112018.ano.swc_root4734415_child4734414.swc
saving  075_WS3_LijLiu_Liuan_05152018.ano.swc_root4067815_child4067814.swc
saving  048_06102018.ano.swc_root1454377_child1454376.swc
saving  saving 078_LiuAn_LiujLiu05282018.ano.swc_root5036468_child5036467.swc 117_ZZ_LijLiu05212018.ano.swc_root26940164_child26940163.swc

saving  039_YJ_LijLiu_05192018.ano.swc_root12147097_child12147096.swc
saving  054_06102018.ano.swc_root5152562_child5152561.swc
saving  048_06102018.ano.swc_root1449425_child1449424.swc
saving  036_HGD_LijLiu_HGD_05232018.ano.swc_root669962_child669961.swc
saving  045_LjLiu_05132018.ano.swc_root8054099_child8054098.swc
saving  063_06102018.ano.swc_root428776_child428775.swc
saving  040_06102018.ano.swc_root1051018_child1051017.swc
saving  021_LiMin_LijLiu_06042018.ano.sw

saving  036_HGD_LijLiu_HGD_05232018.ano.swc_root662240_child662239.swc
saving  113_HGD_LijLiu_05302018.ano.swc_root7140259_child7140258.swc
saving  036_HGD_LijLiu_HGD_05232018.ano.swc_root970518_child970517.swc
saving  112_YJ_LijLiu05242018.ano.swc_root5627699_child5627698.swc
saving  036_HGD_LijLiu_HGD_05232018.ano.swc_root947463_child947462.swc
saving  065_06112018.ano.swc_root4742653_child4742652.swc
saving  021_LiMin_LijLiu_06042018.ano.swc_root6095572_child6095571.swc
saving  037_06082018.ano.swc_root1183003_child1183002.swc
saving  025_ZSJ_LijLiu_06072018.ano.swc_root5302355_child5302354.swc
saving  079_06112018.ano.swc_root7040440_child7040439.swc
saving  036_HGD_LijLiu_HGD_05232018.ano.swc_root970518_child970517.swc
saving  112_YJ_LijLiu05242018.ano.swc_root5627699_child5627698.swc
saving  074_LiuAn_LijLiu_Liuan_05152018.ano.swc_root3824263_child3824262.swc
saving  038_06092018.ano.swc_root2274542_child2274541.swc
saving  025_ZSJ_LijLiu_06072018.ano.swc_root4694679_child4694678

...

saving  079_06112018.ano.swc_root7039631_child7039630.swc
saving  037_06082018.ano.swc_root1177760_child1177759.swc
saving  025_ZSJ_LijLiu_06072018.ano.swc_root4845690_child4845689.swc
saving  118_HGD_LijLiu_05302018.ano.swc_root2004266_child2004259.swc
saving  080_05092018.ano.swc_root1794349_child1794348.swc
saving  120_06112018.ano.swc_root10382381_child10382380.swc
saving  005_LijLiu_05272018.ano.swc_root1375217_child1375216.swc
saving  054_06102018.ano.swc_root4237907_child4237906.swc
saving  080_05092018.ano.swc_root1229385_child1229384.swc
saving  052_06102018.ano.swc_root2802976_child2802975.swc
saving  024_ZSJ_LijLiu_06122018.ano.swc_root1211784_child1211783.swc
saving  037_06082018.ano.swc_root1190986_child1190985.swc
saving  112_YJ_LijLiu05242018.ano.swc_root4061483_child4061482.swc
saving  110_ZWX_LijLiu_06072018.ano.swc_root5502861_child5502860.swc
saving  076_Liuan_05162018.swc_root716163_child716162.swc
saving  038_06092018.ano.swc_root2271058_child2271057.swc
saving  04

saving  080_05092018.ano.swc_root1304095_child1304094.swc
saving  004_ZZ_LiuAn_05302018.ano.swc_root1666126_child1666125.swc
saving  119_z_5010_y_38924_x_18536.ano.swc_root8561451_child8561450.swc
saving  041_YJ_LijLiu_05172018.ano.swc_root6501204_child6501203.swc
saving  saving 063_06102018.ano.swc_root394395_child394394.swc
 080_05092018.ano.swc_root1259191_child1259190.swc
saving  118_HGD_LijLiu_05302018.ano.swc_root2000570_child2000562.swc
saving  005_LijLiu_05272018.ano.swc_root1391330_child1391329.swc
saving  024_ZSJ_LijLiu_06122018.ano.swc_root1192131_child1192130.swc
saving  116_Linus_LijLiu_06042018.ano.swc_root4592277_child4592276.swc
saving  saving 083_LiuAn_LijLiu_06052018.ano.swc_root2280117_child2280116.swc
 021_LiMin_LijLiu_06042018.ano.swc_root6109833_child6109832.swc
saving  039_YJ_LijLiu_05192018.ano.swc_root12145325_child12145324.swc
saving  055_06102018.ano.swc_root6358321_child6358320.swc
saving  066_06112018.ano.swc_root5493464_child5493463.swc
saving  052_0610201

 078_LiuAn_LiujLiu05282018.ano.swc_root5036468_child5036467.swc
saving  081_LiuAn_LijLiu_05302018.ano.swc_root682581_child682580.swc
saving  118_HGD_LijLiu_05302018.ano.swc_root2007322_child2007327.swc
saving  037_06082018.ano.swc_root1186424_child1186423.swc
saving  080_05092018.ano.swc_root1305330_child1305329.swc
saving  037_06082018.ano.swc_root1173751_child1173750.swc
saving  079_06112018.ano.swc_root7038136_child7038135.swc
saving  049_YJ_LijLiu_05242018.ano.swc_root3196443_child3196442.swc
saving  038_06092018.ano.swc_root1846522_child1846521.swc
saving  117_ZZ_LijLiu05212018.ano.swc_root26618503_child26618502.swc
saving saving   103_ZSJ_LijLiu_06122018.ano.swc_root1683758_child1683757.swc
117_ZZ_LijLiu05212018.ano.swc_root26939890_child26939889.swc
saving  025_ZSJ_LijLiu_06072018.ano.swc_root5036033_child5036032.swc
saving  065_06112018.ano.swc_root4849757_child4849756.swc
saving  054_06102018.ano.swc_root5131404_child5131403.swc
saving  005_LijLiu_05272018.ano.swc_root1391330_

saving  066_06112018.ano.swc_root6509229_child6509228.swc
saving  041_YJ_LijLiu_05172018.ano.swc_root6492425_child6492424.swc
saving  118_HGD_LijLiu_05302018.ano.swc_root2007322_child2007327.swc
saving  021_LiMin_LijLiu_06042018.ano.swc_root6095572_child6095571.swc
saving  103_ZSJ_LijLiu_06122018.ano.swc_root1690025_child1690024.swc
saving  078_LiuAn_LiujLiu05282018.ano.swc_root5054306_child5054305.swc
saving  117_ZZ_LijLiu05212018.ano.swc_root26616253_child26616252.swc
saving  034_YJ_LijLiu05302018.ano.swc_root1196100_child1196099.swc
saving  076_Liuan_05162018.swc_root677222_child677221.swc
saving  103_ZSJ_LijLiu_06122018.ano.swc_root1690025_child1690024.swc
saving  063_06102018.ano.swc_root423633_child423632.swc
saving  053__06102018.ano.swc_root3598425_child3598424.swc
saving  035_HGD_LijLiu05182018.ano.swc_root12273221_child12273220.swc
saving  079_06112018.ano.swc_root7057211_child7057210.swc
saving  036_HGD_LijLiu_HGD_05232018.ano.swc_root954128_child954127.swc
saving  048_06102

saving  021_LiMin_LijLiu_06042018.ano.swc_root6109833_child6109832.swc
saving  078_LiuAn_LiujLiu05282018.ano.swc_root5050014_child5050013.swc
saving  082_LiuAn_LijLiu_06042018.ano.swc_root1506868_child1506867.swc
saving  053__06102018.ano.swc_root3870466_child3870465.swc
saving  034_YJ_LijLiu05302018.ano.swc_root1201630_child1201629.swc
saving  saving 103_ZSJ_LijLiu_06072018.ano.swc_root2213109_child2213108.swc 034_YJ_LijLiu05302018.ano.swc_root1201630_child1201629.swc

saving  080_05092018.ano.swc_root1792541_child1792540.swc
saving  038_06092018.ano.swc_root1846522_child1846521.swc
saving  118_HGD_LijLiu_05302018.ano.swc_root2002694_child2002698.swc
saving  115_LijLiu_05272018.ano.swc_root3839791_child3839790.swc
saving  041_YJ_LijLiu_05172018.ano.swc_root6501699_child6501698.swc
saving  018_05092018.ano.swc_root122224_child122204.swc
saving  117_ZZ_LijLiu05212018.ano.swc_root26623796_child26623795.swc
saving  052_06102018.ano.swc_root2819804_child2819803.swc
saving  052_06102018.ano

Note: since we are randomly cropping the node cloud, we will no longer have a node designated as the root node "-1" in each file.